# Extract Job Posts from Indeed

Before extracting job posts from [Indeed](https://www.indeed.com/), make sure you have checked their [robots.txt](https://www.indeed.com/robots.txt) file. 

## Create a table in database

In [14]:
import pandas
import configparser
import psycopg2

Read the database connection info from the config.ini

In [15]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['db']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

Establish a connection to the databas, and create a cursor.

In [16]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

Design the table in SQL

In [17]:
# replace the schema and table name to your schema and table name
table_sql = """
            CREATE TABLE IF NOT EXISTS gp7.indeed
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

create the table

In [18]:
cur.execute(table_sql)
conn.commit()

## Request HTML

[urllib.request](https://docs.python.org/3/library/urllib.request.html) makes simple HTTP requests to visit a web page and get the content via the Python standard library.
Here we define the URL to search job pots about Intelligence analyst.

In [19]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=2'

In [20]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))

# Parese HTML
We can use the inspector tool in browsers to analyze webpages and use [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to extract webpage data.
pip install the beautiful soup if needed. 

In [21]:
!pip install beautifulsoup4

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [22]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

Use the <code> tag.find_all(‘tag_name’, tage_attr = ‘possible_value’)</code> function to return a list of tags where the attribute equals the possible_value.
<ul>Common attributes include:
<li> id</li>
<li> class<span style ="color:red">_</span></li>
</ul>

<ul>Common functions include:
<li> tag.text: return the visible part of the tag</li>
<li> tag.get(‘attribute’): return the value of the attribute of the tag</li>
</ul>

Since all the job posts are in the <code>div tag class = 'jobsearch-Sprep...'</code>, we need to find that div tag from the body tag.

In [23]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [24]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [25]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

## Save Data to Database
Now we find the div tag contains the job posts. We need to identify the job title, company, ratings, reviews, salary, and summary. 
We can save those records to our table in the database.

In [26]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for h2_title in div_row.find_all('h2', class_ = 'title'):
       
        job_title = h2_title.a.text.strip().replace("'","_")
   
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into gp7.indeed(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

## Query the Table

In [27]:
df = pandas.read_sql_query('select * from gp7.indeed', conn) 
df[:] 

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Business/Research Analyst - Diversified Indust...,"Proactive Worldwide, Inc.","Chicago, IL","$70,000 - $80,000 a year",Execute Proactive processes and tools with exc...
1,2,Threat Intelligence Analyst,Ford Motor Company,None,None,This will involve the use of a variety of tool...
2,3,Intelligence Analyst (Remote),CrowdStrike,None,None,"Undergraduate degree, military training or rel..."
3,4,Intelligence Analyst,"Array Information Technology, Inc.",None,None,.Demonstrated ability to create and provide in...
4,5,Lead Global Intelligence Analyst,AlertMedia,None,None,You have previous leadership experience in the...
5,6,Intelligence Analyst,Ashburn Consulting,None,None,Demonstrated ability to create and provide int...
6,7,"Analyst, Intelligence and Activation",W2O,None,None,Assist in 1-2 ongoing analytics reports across...
7,8,Intelligence Analyst,Samaritan Protective Services,None,Up to $25 an hour,"Degree in political science, intelligence stud..."
8,9,Linguist/Language Analyst (Russian/Chinese),National Security Agency,None,"$64,009 - $99,741 a year",Prior military without a degree but with langu...
9,10,Intelligence Analyst I - Incident,WorldAware,None,None,0 to 3 years of work experience in a related f...


In [28]:
cur.close()
conn.close()